In [52]:
import os,requests
import pandas as pd

### tried methods (but failed ones) : 1.  OAuth1(client ids and client secrets) 2. API keys and secrets
### tried best method: beare token 

## read csv and load in 

In [124]:
parent_dir = os.path.dirname(os.getcwd())
tweets_path = os.path.join(parent_dir, "data", "raw", "tweets_hourly.csv")

df= pd.read_csv(tweets_path)
df


,text,id,created_at,edit_history_tweet_ids
0,@eliant_capital The new shiny object is #AI \n...,1665886218951819264,2023-06-06T00:59:59.000Z,['1665886218951819264']
1,RT @StarryNift: 💥StarryNift Citizenship &amp; ...,1665886202795544576,2023-06-06T00:59:56.000Z,['1665886202795544576']
2,RT @StarryNift: 💥StarryNift Citizenship &amp; ...,1665886195279368193,2023-06-06T00:59:54.000Z,['1665886195279368193']
3,RT @StarryNift: 💥StarryNift Citizenship &amp; ...,1665886181748543488,2023-06-06T00:59:51.000Z,['1665886181748543488']
4,RT @AnaLok17197515: originally issued as a mem...,1665886181173624832,2023-06-06T00:59:50.000Z,['1665886181173624832']
...,...,...,...,...
175,RT @MeStar_Game: 🎉 It’s time for a MeStar Repo...,1666233456026611712,2023-06-06T23:59:47.000Z,['1666233456026611712']
176,RT @Venom_ecosystem: WL Giveaway!🚨\nRave game ...,1666233448346861569,2023-06-06T23:59:45.000Z,['1666233448346861569']
177,@cb_doge And #AlphaCoin is the Best #Metaverse...,1666233439593406465,2023-06-06T23:59:43.000Z,['1666233439593406465']
178,RT @sophiaverse: Welcome to SophiaVerse! 🪷\n\n...,1666233428482686980,2023-06-06T23:59:41.000Z,['1666233428482686980']


In [125]:
import re

# Define the cleaning patterns using regular expressions
whitespace = re.compile(r"\s+")
web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
metaverse = re.compile(r"(?i)@metaverse(?=\b)")
user = re.compile(r"(?i)@[a-z0-9_]+")

# Apply data cleaning using regular expressions to the "text" column
df["cleaned_text"] = df["text"].copy()  # Create a copy of the "text" column
df["cleaned_text"] = df["cleaned_text"].str.replace(whitespace, " ")
df["cleaned_text"] = df["cleaned_text"].str.replace(web_address, "")
df["cleaned_text"] = df["cleaned_text"].str.replace(user, "")


Sentiment analysis with NLP

In [126]:
#labelling each tweet with emotions label 
import flair
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

In [127]:
probs = []
sentiments = []
score = []
for i in range(0, len(df)):
    tweet = df.iloc[i, 2]  # Use the correct index of the "text" column
    sentence = flair.data.Sentence(tweet)
    sentiment_model.predict(sentence)
    # Extract sentiment prediction
    probs.append(sentence.labels[0].score)  # Numerical score 0-1
    sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'
    if sentence.labels[0].value == "NEGATIVE":
        score.append(-sentence.labels[0].score)
    else:
        score.append(sentence.labels[0].score)

# Add probability and sentiment predictions to df
df['probability'] = probs
df['sentiment'] = sentiments
df['score'] = score


In [129]:
# 1) Convert "created_at" column to datetime format
df['created_at'] = pd.to_datetime(df['created_at'])

# 2) Keep only "score" and "created_at" columns
df = df[['score', 'created_at']]

# Rename "created_at" column to "date"
df = df.rename(columns={'created_at': 'date'})

# 3) Convert "date" column to the desired format with hourly time series
df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:00')



In [130]:
df_copy = df.copy()

In [131]:
# Calculate the average score per date
average_scores = df.groupby('date')['score'].mean()

# Create a new DataFrame with average scores per date
df_scores = pd.DataFrame({'date': average_scores.index, 'average_score': average_scores.values})